In [ ]:
# give 2 portforlio, the old one and the optimized new one, 
# calculate the stepwise change of the portfolio
# and prioritize the changes by the improvement of the portfolio

cur_investment = [

        {
            "stock": "GILD",
            "quantity": 55,
        },
        {
            "stock": "LLY",
            "quantity": 6.68,
        },
        {
            "stock": "REGN",
            "quantity": 0.5,
        },
        {
            "stock": "STAN.L",
            "quantity": 196.2
        },
        {
            "stock": "601225.SS",
            "quantity": 600
        },
        {
            "stock": "600276.SS",
            "quantity": 300
        },
        {
            "stock": "MRK",
            "quantity": 11.67
        },
        {
            "stock": "HUBB",
            "quantity": 6.56
        },
        {
            "stock": "VRTX",
            "quantity": 10.32
        },
        {
          "stock": "LULU",
            "quantity": 7.18
        },
        {
          "stock": "SMCI",
            "quantity": 99.96
        }
    ]


In [17]:
# compute the portfolio value

from util import get_currency_pair, read_and_filter_exchange_rates

tot_value = 0
for stock in cur_investment:
    # get the price of the stock which is the last line in the stock price file
    file_path = './data/prices/' + stock["stock"] + '.csv'
    with open(file_path) as f:
        lines = f.readlines()
        stock_price = float(lines[-1].split(",")[1])
        # if the price is zero, use the previous price
        if stock_price == 0:
            stock_price = float(lines[-2].split(",")[1])

    if '.' in stock["stock"]:
        stock_suffix = '.' + stock["stock"].split(".")[-1]
        # convert GBX to GBP
        if stock_suffix == '.L':
            stock_price *= 0.01
        exchange_name, needs_inversion, exchange_name_yahoo = get_currency_pair(stock_suffix, "USD")
        df_rate = read_and_filter_exchange_rates(exchange_name, exchange_name_yahoo)
        #print(df_rate)
        rate = df_rate[exchange_name].iloc[-1]

        if needs_inversion:
            rate = 1 / rate

        print(stock["stock"], stock_price, stock_price * rate, stock["quantity"] * stock_price * rate)
        tot_value += stock["quantity"] * stock_price * rate
        stock['value'] = stock["quantity"] * stock_price * rate
    else:
        print(stock["stock"], stock_price, stock["quantity"] * stock_price)
        tot_value += stock["quantity"] * stock_price
        stock['value'] = stock["quantity"] * stock_price


print(tot_value)


GILD 88.11000061035156 4846.050033569336
LLY 735.7899780273438 4915.077053222656
REGN 742.1900024414062 371.0950012207031
STAN.L 9.5 12.007229328155518 2355.8183941841125
601225.SS 23.8799991607666 3.2985702064742095 1979.1421238845257
600276.SS 50.310001373291016 6.949375102586348 2084.8125307759046
MRK 96.9800033569336 1131.756639175415
FRAS.L 7.51 9.492030763626099 4746.015381813049
VRTX 448.4599914550781 4628.107111816406
LULU 298.739990234375 2144.9531298828124
SMCI 27.290000915527344 2727.908491516113
31930.735891061035


In [18]:
# compute the weight
for stock in cur_investment:
    stock['weight'] = stock['value'] / tot_value

cur_investment

[{'stock': 'GILD',
  'quantity': 55,
  'value': 4846.050033569336,
  'weight': 0.1517675649600666},
 {'stock': 'LLY',
  'quantity': 6.68,
  'value': 4915.077053222656,
  'weight': 0.1539293384903987},
 {'stock': 'REGN',
  'quantity': 0.5,
  'value': 371.0950012207031,
  'weight': 0.011621874374796061},
 {'stock': 'STAN.L',
  'quantity': 196.2,
  'value': 2355.8183941841125,
  'weight': 0.0737790197576881},
 {'stock': '601225.SS',
  'quantity': 600,
  'value': 1979.1421238845257,
  'weight': 0.06198235238413606},
 {'stock': '600276.SS',
  'quantity': 300,
  'value': 2084.8125307759046,
  'weight': 0.06529171572771503},
 {'stock': 'MRK',
  'quantity': 11.67,
  'value': 1131.756639175415,
  'weight': 0.03544411387938756},
 {'stock': 'FRAS.L',
  'quantity': 500,
  'value': 4746.015381813049,
  'weight': 0.14863470099797135},
 {'stock': 'VRTX',
  'quantity': 10.32,
  'value': 4628.107111816406,
  'weight': 0.1449420748587269},
 {'stock': 'LULU',
  'quantity': 7.18,
  'value': 2144.953129882

In [19]:
import pandas as pd
import numpy as np
import json

# enumerate the directory and find out the latest file
import os


data_dir = "./processed_data_128"

S = pd.read_pickle(f'{data_dir}/S.pkl')
mu = np.load(f'{data_dir}/mu.npy')
# read the latest json data


latest_file = None
latest_time = 0
for file in os.listdir(data_dir + '/computed_portfolios'):
    if file.endswith(".json"):
        file_time = os.path.getmtime(data_dir + '/computed_portfolios/' + file)
        if file_time > latest_time:
            latest_time = file_time
            latest_file = file

with open(data_dir + '/computed_portfolios/' + latest_file) as f:
    tickers_to_buy = json.load(f)

with open(f'{data_dir}/valid_tickers.txt', 'r') as f:
    valid_tickers = f.read().splitlines()


In [20]:
current_weight = np.zeros(len(valid_tickers))
for stock in cur_investment:
    if stock["stock"] in valid_tickers:
        current_weight[valid_tickers.index(stock["stock"])] = stock["weight"]

In [21]:
import sys


from util import portfolio_log_return, portfolio_return, portfolio_volatility, portfolio_volatility_log_return


print(portfolio_volatility(current_weight, S))
print(portfolio_return(current_weight, mu))
sharpe = portfolio_log_return(current_weight, mu) - portfolio_volatility_log_return(current_weight, S)
print(sharpe)

0.07874108399813194
0.13892809172712012
0.05344711414992402


In [22]:
tickers_to_buy

[{'id': 'RHM.DE', 'weight': 0.019999999999988874},
 {'id': 'BA.L', 'weight': 0.02},
 {'id': 'ABBV', 'weight': 0.02},
 {'id': 'AMGN', 'weight': 0.02},
 {'id': 'AMAT', 'weight': 0.019999999999997076},
 {'id': 'AJG', 'weight': 0.012333925984780137},
 {'id': 'BIIB', 'weight': 0.02},
 {'id': 'AVGO', 'weight': 0.02},
 {'id': 'BRO', 'weight': 0.02},
 {'id': 'CDNS', 'weight': 0.02},
 {'id': 'CPB', 'weight': 0.02},
 {'id': 'CAH', 'weight': 0.02},
 {'id': 'COR', 'weight': 0.02},
 {'id': 'CMG', 'weight': 0.01999999999999876},
 {'id': 'CI', 'weight': 0.02},
 {'id': 'CTAS', 'weight': 0.019506190810842382},
 {'id': 'ED', 'weight': 0.02},
 {'id': 'CTRA', 'weight': 0.02},
 {'id': 'DVN', 'weight': 0.002592547144342377},
 {'id': 'LLY', 'weight': 0.02},
 {'id': 'EQT', 'weight': 0.0034784194072669944},
 {'id': 'FICO', 'weight': 0.02},
 {'id': 'GILD', 'weight': 0.02},
 {'id': 'HOLX', 'weight': 0.013859344593091457},
 {'id': 'HUBB', 'weight': 0.02},
 {'id': 'HUM', 'weight': 0.02},
 {'id': 'JBL', 'weight': 0

In [23]:
def update_weight(weights_attr, unique_ids):
  weights = np.zeros(len(unique_ids))
  for asset in weights_attr:
    asset_id = asset['id']
    asset_weight = asset['weight']
    asset_index = unique_ids.index(asset_id)
    weights[asset_index] = asset_weight
  return weights

In [24]:
tickers_to_buy

[{'id': 'RHM.DE', 'weight': 0.019999999999988874},
 {'id': 'BA.L', 'weight': 0.02},
 {'id': 'ABBV', 'weight': 0.02},
 {'id': 'AMGN', 'weight': 0.02},
 {'id': 'AMAT', 'weight': 0.019999999999997076},
 {'id': 'AJG', 'weight': 0.012333925984780137},
 {'id': 'BIIB', 'weight': 0.02},
 {'id': 'AVGO', 'weight': 0.02},
 {'id': 'BRO', 'weight': 0.02},
 {'id': 'CDNS', 'weight': 0.02},
 {'id': 'CPB', 'weight': 0.02},
 {'id': 'CAH', 'weight': 0.02},
 {'id': 'COR', 'weight': 0.02},
 {'id': 'CMG', 'weight': 0.01999999999999876},
 {'id': 'CI', 'weight': 0.02},
 {'id': 'CTAS', 'weight': 0.019506190810842382},
 {'id': 'ED', 'weight': 0.02},
 {'id': 'CTRA', 'weight': 0.02},
 {'id': 'DVN', 'weight': 0.002592547144342377},
 {'id': 'LLY', 'weight': 0.02},
 {'id': 'EQT', 'weight': 0.0034784194072669944},
 {'id': 'FICO', 'weight': 0.02},
 {'id': 'GILD', 'weight': 0.02},
 {'id': 'HOLX', 'weight': 0.013859344593091457},
 {'id': 'HUBB', 'weight': 0.02},
 {'id': 'HUM', 'weight': 0.02},
 {'id': 'JBL', 'weight': 0

In [25]:
opti_weight = update_weight(tickers_to_buy, valid_tickers)

print(portfolio_volatility(opti_weight, S))
print(portfolio_return(opti_weight, mu))
sharpe = portfolio_log_return(opti_weight, mu) - portfolio_volatility_log_return(opti_weight, S)
print(sharpe)

0.06008907218513227
0.14742587049821776
0.0771376862301945


In [26]:
# to find the steps to optimize my current portfolio to the optimized portfolio
# we need to find the difference between the optimized portfolio and the current portfolio
# and then prioritize the changes by the improvement of the portfolio

old_portfolio = pd.DataFrame(cur_investment)
optimized_portfolio = pd.DataFrame(tickers_to_buy)

# Merge the two dataframes to compare weights
comparison = pd.merge(old_portfolio, optimized_portfolio, left_on='stock', right_on='id', how='outer')
comparison['stock'] = comparison['stock'].fillna(comparison['id'])
comparison.fillna(0, inplace=True)  # Assuming no stock means 0 weight
comparison['weight_difference'] = comparison['weight_y'] - comparison['weight_x']

#*comparison['new_quantity'] = comparison['quantity'] * comparison['weight_y'] / comparison['weight_x'] if comparison['weight_x'] != 0 else 0
print(comparison[['stock','weight_x', 'weight_y', 'weight_difference', 'quantity']])

        stock  weight_x  weight_y  weight_difference  quantity
0        GILD  0.151768  0.020000          -0.131768     55.00
1         LLY  0.153929  0.020000          -0.133929      6.68
2        REGN  0.011622  0.020000           0.008378      0.50
3      STAN.L  0.073779  0.000000          -0.073779    196.20
4   601225.SS  0.061982  0.020000          -0.041982    600.00
5   600276.SS  0.065292  0.008117          -0.057175    300.00
6         MRK  0.035444  0.020000          -0.015444     11.67
7      FRAS.L  0.148635  0.000000          -0.148635    500.00
8        VRTX  0.144942  0.020000          -0.124942     10.32
9        LULU  0.067175  0.020000          -0.047175      7.18
10       SMCI  0.085432  0.020000          -0.065432     99.96
11     RHM.DE  0.000000  0.020000           0.020000      0.00
12       BA.L  0.000000  0.020000           0.020000      0.00
13       ABBV  0.000000  0.020000           0.020000      0.00
14       AMGN  0.000000  0.020000           0.020000   

In [27]:
comparison

,stock,quantity,value,weight_x,id,weight_y,weight_difference
0,GILD,55.00,4846.050034,0.151768,GILD,0.020000,-0.131768
1,LLY,6.68,4915.077053,0.153929,LLY,0.020000,-0.133929
2,REGN,0.50,371.095001,0.011622,REGN,0.020000,0.008378
3,STAN.L,196.20,2355.818394,0.073779,0,0.000000,-0.073779
4,601225.SS,600.00,1979.142124,0.061982,601225.SS,0.020000,-0.041982
5,600276.SS,300.00,2084.812531,0.065292,600276.SS,0.008117,-0.057175
6,MRK,11.67,1131.756639,0.035444,MRK,0.020000,-0.015444
7,FRAS.L,500.00,4746.015382,0.148635,0,0.000000,-0.148635
8,VRTX,10.32,4628.107112,0.144942,VRTX,0.020000,-0.124942
9,LULU,7.18,2144.953130,0.067175,LULU,0.020000,-0.047175


In [28]:
def portfolio_volatility_log_return(weights, covariance):
    return np.sqrt(np.dot(weights.T, np.dot(covariance, weights)))

def portfolio_var_log_return(weights, covariance):
    return np.dot(weights.T, np.dot(covariance, weights))

def portfolio_log_return(weights, returns, allow_short=False):
    return np.sum(np.abs(returns)*weights) if allow_short else np.sum(returns*weights)

In [29]:
# Initialize the weights array

current_weight = np.zeros(len(valid_tickers))
for stock in cur_investment:
    if stock["stock"] in valid_tickers:
        current_weight[valid_tickers.index(stock["stock"])] = stock["weight"]


original_sharpe = portfolio_log_return(current_weight, mu) - portfolio_volatility_log_return(current_weight, S)
print(f'Original Sharpe ratio: {original_sharpe}')
# Optimize one stock at a time and adjust other weights proportionally

new_weights = current_weight.copy()
for i in range(20):
    results = []
    print(f'Starting iteration {i} optimization...')
    for index, row in comparison.iterrows():
        new_weights_try = new_weights.copy()
        stock_name = comparison.loc[index, 'stock']
        stock_idx = valid_tickers.index(stock_name)
        # Adjust the targeted stock weight
        new_weights_try[stock_idx] = comparison.loc[index, 'weight_y']
        # Adjust the rest to ensure total sum remains 1
        total_adjusted_weights = np.sum(new_weights_try) - new_weights_try[stock_idx]
        scale_factor = (1 - new_weights_try[stock_idx]) / total_adjusted_weights
        for j in range(len(new_weights_try)):
            if j != stock_idx:
                new_weights_try[j] *= scale_factor

        new_sharpe = portfolio_log_return(new_weights_try, mu) - portfolio_volatility_log_return(new_weights_try, S)
        results.append((comparison.loc[index, 'stock'], new_sharpe, new_sharpe - original_sharpe, new_weights_try, comparison.loc[index, 'weight_difference']))

    results.sort(key=lambda x: x[2], reverse=True)

    stock_name = results[0][0]
    stock_idx = valid_tickers.index(stock_name)
    # Adjust the targeted stock weight
    new_weights = results[0][3]
    new_sharpe = results[0][1]
    # remove the row from comparison
    comparison = comparison[comparison['stock'] != stock_name]

    original_sharpe = new_sharpe

    print(f'Iteration {i} optimization result:')
    print(f'Optimized ticker: {stock_name}')
    print(f'Sharpe ratio: {results[0][1]}')
    print(f'Improvement: {results[0][2]}')
    print(f'Weight difference: {results[0][4]}')
    


Original Sharpe ratio: 0.05344711414992402
Starting iteration 0 optimization...
Iteration 0 optimization result:
Optimized ticker: FRAS.L
Sharpe ratio: 0.07700833300365142
Improvement: 0.023561218853727403
Weight difference: -0.14863470099797135
Starting iteration 1 optimization...
Iteration 1 optimization result:
Optimized ticker: 600519.SS
Sharpe ratio: 0.08001620203769322
Improvement: 0.0030078690340417935
Weight difference: 0.02
Starting iteration 2 optimization...
Iteration 2 optimization result:
Optimized ticker: JKHY
Sharpe ratio: 0.08276406503138505
Improvement: 0.002747862993691827
Weight difference: 0.02
Starting iteration 3 optimization...
Iteration 3 optimization result:
Optimized ticker: 600031.SS
Sharpe ratio: 0.08514477466688562
Improvement: 0.0023807096355005775
Weight difference: 0.02
Starting iteration 4 optimization...
Iteration 4 optimization result:
Optimized ticker: HOLX
Sharpe ratio: 0.08665055209447266
Improvement: 0.001505777427587035
Weight difference: 0.01385

In [ ]:
optimized_portfolio